In [2]:
import torch
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
import hashlib


/home/mleshashi/thesis-sharma/sraEnv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Correctly formatted URL for a local Elasticsearch instance
es = Elasticsearch(["http://localhost:9200"])

In [4]:
# Check the health of the cluster
health = es.cluster.health()
print(health)

{'cluster_name': 'elasticsearch', 'status': 'yellow', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 1, 'active_shards': 1, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 1, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 50.0}


****Finding the Most Relevant Topics using Pew Dataset****

In [5]:
# Ensure the entire text in each cell is displayed without truncation
pd.set_option('display.max_colwidth', None)

# Adjusting display width might not be necessary depending on your pandas version, but it's here for completeness
pd.set_option('display.width', None)

# Load your DataFrame
pew = pd.read_csv('../dataset/pew_dataset/metadata.csv')
columns = ['title','caption']

# Filtering the DataFrame to include only the specified columns
pew_df = pew[columns]

In [6]:
pew_df.head()

,title,caption
0,"Foreign-born population in the United States, 1850-2018","The foreign-born population residing in the U.S. reached a record 44.8 million, or 13.7% of the U.S. population, in 2018. This immigrant population has more than quadrupled since the 1960s, when the 1965 Immigration and Naturalization Act took effect. Though growth has begun to slow in recent years, the number of immigrants living in the United States is projected to almost double by 2065."
1,"English proficiency among U.S. immigrants, 1980-2018","Since 1980, the share of immigrants who are proficient in English (those who speak only English at home or speak English at least “very well”) has declined, though it has increased slightly in recent years. This decline has been driven entirely by those who speak only English at home, which fell from 30% of immigrants ages 5 and older in 1980 to 17% in 2018. The share who speaks English “very well,” meanwhile, has increased slightly, from 27% to 37% over the same time period."
2,"Languages spoken among U.S. immigrants, 2018","Among the nation’s immigrants, Spanish is by far the most spoken non-English language (42% of immigrants say they speak Spanish at home), but it is not the only non-English language spoken by immigrants. Some 6% of immigrants speak Chinese (including Mandarin and Cantonese), 5% speak Hindi or a related language, 4% speak Filipino or Tagalog, 3% speak Vietnamese, 3% speak French and 2% speak Dravidian."
3,"Hispanic population in the U.S., 2000-2017","There were nearly 60 million Latinos in the United States in 2017, accounting for approximately 18% of the total U.S. population. In 1980, with a population of 14.8 million, Hispanics made up just 6.5% of the total U.S. population. For more, read the accompanying blog post, “Key facts about U.S. Hispanics and their diverse heritage.” For facts on the foreign-born population in the United States, see our profile on U.S. immigrants."
4,Weekly broadcast audience for top 20 NPR-affiliated radio stations,"The top 20 NPR-affiliated public radio stations (by listenership) had on average a total weekly listenership of about 11 million in 2018, about the same as in 2017. (This includes listeners of NPR programming as well as original or other syndicated content aired on these stations.)"


In [7]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [8]:
index_name = "documents"

In [9]:
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body={
        "mappings": {
            "properties": {
                "title": {"type": "text"},
                "content": {"type": "text"},
                "embedding": {"type": "dense_vector", "dims": 384}
            }
        }
    })

In [10]:
# Check if the index exists
exists = es.indices.exists(index=index_name)
print(f"Index Exists: {exists}")

# Get index information (settings and mappings)
if exists:
    index_info = es.indices.get(index=index_name)
    print(index_info)

Index Exists: True
{'documents': {'aliases': {}, 'mappings': {'properties': {'content': {'type': 'text'}, 'embedding': {'type': 'dense_vector', 'dims': 384, 'index': True, 'similarity': 'cosine'}, 'title': {'type': 'text'}}}, 'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'documents', 'creation_date': '1709672865336', 'number_of_replicas': '1', 'uuid': '4U_yVTmYTziNKUy-6RPndg', 'version': {'created': '8500010'}}}}}


In [11]:
for _, row in pew_df.iterrows():
    # Concatenate title and caption with a space or some delimiter
    combined_text = row['title'] + ". " + row['caption']
    
    # Generate embedding for the combined text
    embedding = model.encode(combined_text).tolist()

    # Generate a unique ID for the document using a hash of the title and caption
    unique_id = hashlib.sha256(combined_text.encode('utf-8')).hexdigest()
    
    # Index the document with the combined embedding and use the unique_id as the document ID
    es.index(index=index_name, id=unique_id, body={
        "title": row['title'],
        "content": row['caption'],
        "embedding": embedding
    })

In [12]:
doc_count = es.count(index=index_name)['count']
print(f"Number of documents in the index: {doc_count}")

Number of documents in the index: 1480


In [29]:
topics_df= pd.read_csv('../dataset/topics.csv')

In [30]:
topics_df

,Topic
0,Should teachers get tenure?
1,Is vaping with e-cigarettes safe?
2,Should insider trading be allowed?
3,Should corporal punishment be used in schools?
4,Should social security be privatized?
...,...
95,Should government spending be reduced?
96,Should children beauty contests be banned?
97,Should the US Electoral college be abolished?
98,Should social networks be banned?


In [31]:
topics_embeddings = [model.encode(topic) for topic in topics_df['Topic'].values]

In [41]:
# Assuming initialization and index creation as before

topics_df= pd.read_csv('../dataset/topics.csv')

# Dictionary to hold total similarity scores for each topic
topic_similarity_scores = {}

for _, row in topics_df.iterrows():
    topic = row['Topic']
    topic_embedding = model.encode(topic).tolist()
    script_query = {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                "params": {"query_vector": topic_embedding}
            }
        }
    }
    
    response = es.search(index=index_name, body={
        "size": 2000,  # You might want to adjust this size based on your needs
        "query": script_query,
        "_source": {"excludes": ["embedding"]}
    })

    # Sum up the scores of all hits for this topic
    total_score = sum(hit['_score'] for hit in response['hits']['hits'])
    topic_similarity_scores[topic] = total_score

In [44]:
# Rank topics by their total similarity score
sorted_topics = sorted(topic_similarity_scores.items(), key=lambda x: x[1], reverse=True)

# Output the top 10 topics based on cumulative similarity scores
print("Top 10 Relevant Topics Based on Similarity Score:")
for topic, score in sorted_topics[:10]:
    print(f"Topic: {topic}, Total Similarity Score: {score}")

Top 10 Relevant Topics Based on Similarity Score:
Topic: Are social networking sites good for our society?, Total Similarity Score: 1695.5371561500006
Topic: Should we imprison fewer people?, Total Similarity Score: 1677.8149067599988
Topic: Are gender or racial quotas effective?, Total Similarity Score: 1661.0595735099998
Topic: Does legal prostitution increase human trafficking?, Total Similarity Score: 1659.169556260001
Topic: Should the press be subsidized?, Total Similarity Score: 1644.0354750100005
Topic: Are social media platforms doing enough to prevent cyberbullying?, Total Similarity Score: 1640.3085283999994
Topic: Do violent video games contribute to youth violence?, Total Similarity Score: 1639.0179188699997
Topic: Should the voting age be lowered?, Total Similarity Score: 1638.7318199699987
Topic: Should everyone get a universal basic income?, Total Similarity Score: 1635.7151426099995
Topic: Should social networks be banned?, Total Similarity Score: 1635.01015242
